***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-051b  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import struct
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  5 10:41 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'idx316179928316366087idx' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh
postings_small_anchor_text_gcp/0_000.bin
postings_small_anchor_text_gcp/0_posting_locs.pickle
postings_small_anchor_text_gcp/102_000.bin
postings_small_anchor_text_gcp/102_posting_locs.pickle
postings_small_anchor_text_gcp/103_000.bin
postings_small_anchor_text_gcp/103_posting_locs.pickle
postings_small_anchor_text_gcp/105_000.bin
postings_small_anchor_text_gcp/105_posting_locs.pickle
postings_small_anchor_text_gcp/106_000.bin
postings_small_anchor_text_gcp/106_posting_locs.pickle
postings_small_anchor_text_gcp/107_000.bin
postings_small_anchor_text_gcp/107_posting_locs.pickle
postings_small_anchor_text_gcp/108_000.bin
postings_small_anchor_text_gcp/108_posting_locs.pickle
postings_small_anchor_text_gcp/109_000.bin
postings_small_anchor_text_gcp/109_posting_locs.pickle
postings_small_anchor_text_gcp/113_000.bin
postings_small_anchor_text_gcp/113_posting_locs.pickle
postings_small_anchor_text_gcp/119_000.bin
postings_small_anchor_text_gcp/119_posting_locs.pickle
postings_

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
# full_path = "gs://wikidata_preprocessed/*"
# parquetFile = spark.read.parquet(full_path)
# doc_text_pairs = parquetFile.select("anchor", "id").rdd

In [9]:
def id_to_anchor_text(id_and_text):
  return [(id,text) for id, text in id_and_text]

def reduce_anchor_text(list_of_text):
  return " ".join(list_of_text)

####################COMMENT OUT WHEN CALCULATING ANCHOR TEXT INDEX ###########################################
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
x = pages_links.flatMap(lambda x : id_to_anchor_text(x[1]))
doc_text_pairs = x.groupByKey().mapValues(reduce_anchor_text).map(lambda x:(x[1],x[0]))


We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

In [ ]:
INDEX_FOLDER = "body_two_word_index"

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [13]:
from nltk.stem.porter import *

stemmer = PorterStemmer()


english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

query_terms = ['python', 'data', 'scienc', 'migrain', 'chocol', 'how', 'to', 'make', 'pasta', 'doe', 'pasta', 'have', 'preserv', 'how', 'googl', 'work', 'what', 'is', 'inform', 'retriev', 'nba', 'yoga', 'how', 'to', 'not', 'kill', 'plant', 'mask', 'black', 'friday', 'whi', 'do', 'men', 'have', 'nippl', 'rubber', 'duck', 'michelin', 'what', 'to', 'watch', 'best', 'marvel', 'movi', 'how', 'tall', 'is', 'the', 'eiffel', 'tower', 'where', 'doe', 'vanilla', 'flavor', 'come', 'from', 'best', 'ice', 'cream', 'flavour', 'how', 'to', 'tie', 'a', 'tie', 'how', 'to', 'earn', 'money', 'onlin', 'what', 'is', 'critic', 'race', 'theori', 'what', 'space', 'movi', 'wa', 'made', 'in', '1992', 'how', 'to', 'vote', 'googl', 'trend', 'dim', 'sum', 'ted', 'fairi', 'tale']

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  #ADDED stopwords removal

  tokens = [stemmer.stem(token) for token in tokens if token not in all_stopwords]
  # YOUR CODE HERE
  count = Counter(tokens)
  if len(count)==0:
      return []
  #### ADDED doc_len and max_tf to posting lists
  doc_len = len(tokens)
  max_tf = count.most_common(1)[0][1]
  #return [(word,(id,tf,max_tf, doc_len)) for word,tf in count.items() if word not in all_stopwords]
  return [(word,(id,tf,max_tf, doc_len)) for word,tf in count.items() if word not in all_stopwords and word in query_terms]


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key = lambda x:x[0])


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.mapValues(lambda x: len(x))


def word_posting2bucket_and_wpl(word, posting_list):
  return token2bucket_id(word),(word,posting_list)

def write_bucket_to_disk(bucket_id, list_of_words_postings_tuples):
  word2posting_locs = InvertedIndex().write_a_posting_list((bucket_id, list_of_words_postings_tuples),bucket_name) 
  return word2posting_locs

#########CALCULATE CORPUS AVERAGE DOCUMENT LENGTH#####################
def tokenize_remove_stopwords(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  #ADDED stopwords removal
  tokens = [token for token in tokens if token not in all_stopwords]
  return id, len(tokens)

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  bucket2posting_lists=postings.map(lambda x: word_posting2bucket_and_wpl(x[0],x[1])).groupByKey()#.mapValues().reduce_word_counts)
  x= bucket2posting_lists.map(lambda x: write_bucket_to_disk(x[0],x[1]))
  return x

In [14]:
# doc_len = doc_text_pairs.map(lambda x:tokenize_remove_stopwords(x[0],x[1]))
# AVGDL = doc_len.map(lambda x: x[1]).mean()

In [15]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

In [31]:
def is_it_in(x):
    return x[0]=="trend"
    #return (x[0] in ["dim","flavour","flavor","vanilla","eiffel","michelin","nippl","pasta"])

p = postings_filtered.filter(lambda x: is_it_in(x))

p.take(1)[0]


('trend',
 [(9223, 1, 6677, 13033),
  (11657, 3, 1394, 3257),
  (11720, 1, 254, 826),
  (12905, 3, 522, 1399),
  (23053, 1, 641, 1731),
  (25459, 1, 3405, 9221),
  (26094, 2, 1854, 6739),
  (26332, 1, 117, 382),
  (31829, 1, 41, 184),
  (54738, 1, 138, 481),
  (59551, 1, 2597, 8071),
  (62529, 1, 4873, 17078),
  (63082, 66, 176, 653),
  (64699, 1, 32, 191),
  (67630, 1, 37, 125),
  (86350, 1, 50, 63),
  (99260, 1, 626, 1834),
  (112577, 1, 211, 558),
  (118450, 1, 821, 1940),
  (145171, 1, 203, 415),
  (163152, 1, 192, 896),
  (166867, 1, 123, 304),
  (167742, 1, 156, 237),
  (168918, 1, 323, 1404),
  (171323, 8, 71, 246),
  (195521, 1, 537, 1355),
  (198629, 3, 34, 144),
  (207085, 1, 7914, 33115),
  (220964, 1, 781, 2327),
  (228568, 1, 258, 715),
  (327452, 1, 5651, 9225),
  (330217, 1, 1392, 5136),
  (336776, 36, 36, 94),
  (339421, 1, 103, 426),
  (374327, 6, 259, 713),
  (375540, 1, 715, 1851),
  (402982, 6, 333, 1048),
  (406624, 1, 434, 1445),
  (444068, 1, 49, 152),
  (450285,

In [49]:
# def get_max_from_4(tup):
#     maximum = tup[0]
#     for t in tup:
#         if t<maximum:
#             maximum = t
            
#     return maximum



# p.flatMap(lambda x: x[1]).map(get_max_from_4).min()

1

In [ ]:
# test index construction time
#assert index_const_time < 60*120

In [16]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=FOLDER_NAME):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [17]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.AVGDL = 79.8183118653278
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/{FOLDER_NAME}/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  2.2 KiB/  2.2 KiB]                                                
Operation completed over 1 objects/2.2 KiB.                                      


In [18]:
!gsutil ls -lh $index_dst

  2.18 KiB  2022-01-05T14:02:19Z  gs://idx316179928316366087idx/postings_small_anchor_text_gcp_2/index.pkl
TOTAL: 1 objects, 2229 bytes (2.18 KiB)


In [ ]:
def is_it_in(x):
    return x[0]=="trend"
    #return (x[0] in ["dim","flavour","flavor","vanilla","eiffel","michelin","nippl","pasta"])

p = postings_filtered.filter(lambda x: is_it_in(x))

p.take(1)[0]

In [32]:
import struct

def fix_file(pl, offset, blob_number):
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(f"postings_small_anchor_text_gcp_2/{blob_number}_000.bin")    
    if offset==0:
        with blob.open("wb") as f:
            b = b''.join([struct.pack("IIII", doc_id, tf, max_tf, doc_len) for doc_id, tf, max_tf, doc_len in pl])
            f.write(b)
    else:
        with blob.open("rb") as f:
            old_bytes = f.read(offset)
        with blob.open("wb") as f:
            b = b''.join([struct.pack("IIII", doc_id, tf, max_tf, doc_len) for doc_id, tf, max_tf, doc_len in pl])
            b= old_bytes+b
            f.write(b)

all_words_pls = p.take(1)
for word, pl in all_words_pls:
#     pl = p.take(1)[0][1]
    b_id = token2bucket_id(word)
    offset = inverted.posting_locs[word][0][1]

    print(word)
    print(b_id)
    print(offset)
    fix_file(pl, offset, b_id)




trend
27
1536


In [29]:
lst = ["dim","flavour","flavor","vanilla","eiffel","michelin","nippl","pasta"]

for word in lst:
    print(f"{word} : {inverted.posting_locs[word]}")

dim : [('40_000.bin', 425360)]
flavour : [('71_000.bin', 0)]
flavor : [('68_000.bin', 0)]
vanilla : [('50_000.bin', 0)]
eiffel : [('106_000.bin', 0)]
michelin : [('54_000.bin', 0)]
nippl : [('67_000.bin', 0)]
pasta : [('27_000.bin', 0)]


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
def create_references(source_id, dest_list):
  return set([(source_id,dest_id) for dest_id,text in dest_list]) #change to set andd removed distinct at (1)

def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------1
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  # YOUR CODE HERE
  edges = pages.flatMap(lambda x: create_references(x[0],x[1])) # removed distinct (1)
  vertices = edges.flatMap(lambda x: [(x[0],),(x[1],)]).distinct()
  return edges, vertices

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750|  9913.72878216077|
|  10568| 5385.349263642033|
|  32927|5282.0815757652745|
|  30680| 5128.233709604118|
|5843419| 4957.567686263867|
|  68253| 4769.278265355157|
|  31717|4486.3501805483065|
|  11867| 4146.414650912769|
|  14533| 3996.466440885499|
| 645042|  3531.62708980374|
|  17867|3246.0983906041397|
|5042916|2991.9457391661763|
|4689264| 2982.324883041745|
|  14532|2934.7468292031685|
|  25391|2903.5462235133978|
|   5405| 2891.416329154635|
|4764461|2834.3669873326585|
|  15573| 2783.865118158837|
|   9316| 2782.039646413768|
|8569916|2775.2861918400135|
+-------+------------------+
only showing top 20 rows



In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://316366087_bucket/postings_gcp


In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 2.67 
print(f'I used {cost} USD credit during the course of this assignment')

I used 2.67 USD credit during the course of this assignment


**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag